#### Nombre: **04_modelo_raza**
#### Autores: _Sergio Cañón  Laiz y Ignacio Ruiz de Zuazu Echavarría_
#### CUNEF
#### 15/01/2021


# Librerías

In [15]:
# importamos librerías 

import numpy as np  # álgebra lineal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import glob  # pathnames
import cv2  # lectura y procesamiento de las imagenes

import matplotlib.pyplot as plt  # gráficos
import matplotlib.image as mpimg

import os  # lectura de directorios

import keras 
from keras.layers import *
from keras.models import *
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras import optimizers
import time #tiempos de ejecución
from random import shuffle
from keras.preprocessing import image
import imageio


from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.optimizers import SGD
import json

# Importación de datos

In [8]:
categorical_labels_race = np.load(r'..\data\04_data_array\categorical_labels_race.npy')

In [9]:
X = np.load(r'..\data\04_data_array\data_32_32_rgb.npy')

In [12]:
size = len(X)
print("La longuitud del los datos son de:", size)

La longuitud del los datos son de: 24099


Usaremos un 30% de las iamgenes como test y el 70% como train.


In [13]:
#creacion train y test set

(x_train, y_train), (x_test, y_test) = (X[:15008],categorical_labels_race[:15008]) , (X[15008:] , categorical_labels_race[15008:])
(x_valid , y_valid) = (x_test[:7000], y_test[:7000])
(x_test, y_test) = (x_test[7000:], y_test[7000:])


In [14]:
print("Número de imagenes en el train:",x_train.shape[0])
print("Número de imagenes en el test:",x_test.shape[0])

Número de imagenes en el train: 15008
Número de imagenes en el test: 2091


# Model Building

# SGD Model

In [18]:
model_sgd = tf.keras.Sequential()

# Primera capa de la CNN. El imput debe ajsutarse al tamañ ode la imagen (32 x 32 y 3 de los colores)

model_sgd.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', input_shape=(32,32,3))) 
model_sgd.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model_sgd.add(tf.keras.layers.Dropout(0.3))

model_sgd.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu')) 
model_sgd.add(tf.keras.layers.MaxPooling2D(pool_size=2))

# drop out= eliminamos el 30% de las imagenes para evitar overfitting
model_sgd.add(tf.keras.layers.Dropout(0.3))

model_sgd.add(tf.keras.layers.Flatten())
model_sgd.add(tf.keras.layers.Dense(256, activation='relu'))

# drop out= eliminamos la mitad de las imagenes para evitar overfitting
model_sgd.add(tf.keras.layers.Dropout(0.5)) 


# nuestro data set tiene 5 opciones de raza luego la última capa = 5

model_sgd.add(tf.keras.layers.Dense(5, activation='softmax')) 


# Resumen del modelo
model_sgd.summary()

model_sgd.compile(loss='categorical_crossentropy',
             optimizer=SGD(),
             metrics=['accuracy','AUC','categorical_accuracy'])

h_sgd = model_sgd.fit(x_train,
         y_train,
         batch_size=100,
         epochs=25,
         validation_data=(x_valid, y_valid))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 128)       1664      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 128)       65664     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 8, 8, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)             

In [29]:
model_sgd_race_rgb_history = h_sgd.history
json.dump(model_sgd_race_rgb_history, open("../models/model_sgd_race_rgb_history.json", 'w'))

In [21]:
model_sgd_gender.save('../models/model_sgd_race_rgb.h')

INFO:tensorflow:Assets written to: ../models/model_sgd_race_rgb.h\assets


In [ ]:
np.save('x_test_raza.npy', x_test)
np.save('y_test_raza.npy', y_test)


# ADAM Model

In [32]:
model_adam_race = tf.keras.Sequential()

# Primera capa de la CNN. El imput debe ajsutarse al tamañ ode la imagen (32 x 32 y 3 de los colores)
model_adam_race.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', input_shape=(32,32,3))) 
model_adam_race.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model_adam_race.add(tf.keras.layers.Dropout(0.3))

model_adam_race.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu')) 
model_adam_race.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model_adam_race.add(tf.keras.layers.Dropout(0.3))

model_adam_race.add(tf.keras.layers.Flatten())
model_adam_race.add(tf.keras.layers.Dense(256, activation='relu'))
model_adam_race.add(tf.keras.layers.Dropout(0.5))

model_adam_race.add(tf.keras.layers.Dense(5, activation='softmax'))

# Take a look at the model summary
model_adam_race.summary()

model_adam_race.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy','AUC','categorical_accuracy'])

h_model_adam_race = model_adam_race.fit(x_train,
         y_train,
         batch_size=100,
         epochs=25,
         validation_data=(x_valid, y_valid))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 128)       1664      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 128)       65664     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 8, 8, 128)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 8192)             

In [33]:
model_adam_race_history = h_model_adam_race.history
json.dump(model_adam_race_history, open("../models/model_adam_race_history.json", 'w'))

In [34]:
model_adam_race.save('../models/model_adam_race.h')

INFO:tensorflow:Assets written to: ../models/model_adam_race.h\assets
